# PKS Vorverarbeitung
Dieses Notebook verarbeitet die Polizeiliche Kriminalstatistik (PKS).

Ablauf:
1. Excel-Rohdaten in CSV umwandeln (Header-Bereinigung)
2. CSVs mit Spark laden und Spalten bereinigen
3. Deutsche + Nichtdeutsche joinen
4. Als `pks_complete.parquet` speichern

In [ ]:
import sys
import os
import pandas as pd

sys.path.append(os.path.abspath(".."))
from src.config_local import get_spark_session
from pyspark.sql.functions import col

spark = get_spark_session("PKS_Vorverarbeitung")

RAW_DIR = "../data/raw/"
PROC_DIR = "../data/processed/"
os.makedirs(PROC_DIR, exist_ok=True)

Excel -> CSV

Die PKS-Rohdaten liegen als `.xlsx` vor. Die ersten Zeilen enthalten Metadaten, der eigentliche Header beginnt bei der Zeile mit "Schlüssel".

In [ ]:
PKS_FILES = [
    "ZR-TV-04-T40-TV-insg-deutsch_xls.xlsx", 
    "ZR-TV-07-T50-TV-insg-nichtdeutsch_xls.xlsx"
]

for file in PKS_FILES:
    input_path = os.path.join(RAW_DIR, file)
    output_path = os.path.join(RAW_DIR, file.replace(".xlsx", ".csv"))
    
    if not os.path.exists(input_path):
        print(f"Datei nicht gefunden: {input_path}")
        continue
    
    print(f"Konvertiere {file} ...")
    
    # Alles als String lesen, Formatierungsfehler vermeiden
    df = pd.read_excel(input_path, header=None, dtype=str)
    
    # Header beginnt immer bei "Schlüssel" (PKS-Standard)
    start_row = df[df[0] == "Schlüssel"].index[0]
    df_clean = pd.read_excel(input_path, header=start_row)
    
    df_clean.to_csv(output_path, index=False, sep=";", encoding="utf-8")
    print(f" -> {output_path}")

print("CSV-Konvertierung abgeschlossen.")

CSVs laden und bereinigen

Die Spalte `Tatver-dächtige  insgesamt` wird zu `anzahl` umbenannt. Jahr und Anzahl werden über Double nach Int gecastet, weil Excel-Export teilweise Werte wie `"1987.0"` erzeugt.

In [ ]:
def load_and_clean_pks(filename):
    """Lädt eine PKS-CSV und gibt ein bereinigtes DataFrame zurück."""
    df = spark.read.option("header", "true") \
                   .option("delimiter", ";") \
                   .csv(f"{RAW_DIR}{filename}")
    
    # Spaltenname variiert je nach Excel-Export
    if "Tatver-dächtige  insgesamt" in df.columns:
        df = df.withColumnRenamed("Tatver-dächtige  insgesamt", "anzahl")
    
    # Double -> Int Cast, weil Excel "1987.0" statt "1987" exportiert
    df_clean = df.select(
        col("Schlüssel").alias("schluessel"),
        col("Straftat").alias("straftat"),
        col("Jahr").cast("double").cast("int").alias("jahr"),
        col("anzahl").cast("double").cast("int").alias("anzahl")
    )
    
    # Footer-Zeilen und ungültige Jahre rausfiltern
    df_clean = df_clean.filter((col("jahr").isNotNull()) & (col("jahr") > 1900))
    return df_clean

# Beide Dateien laden
df_de = load_and_clean_pks("ZR-TV-04-T40-TV-insg-deutsch_xls.csv")
df_nd = load_and_clean_pks("ZR-TV-07-T50-TV-insg-nichtdeutsch_xls.csv")

print(f"Deutsche:       {df_de.count()} Zeilen")
print(f"Nichtdeutsche:  {df_nd.count()} Zeilen")
df_de.show(5)

Join und Speichern

In [ ]:
# Anzahl-Spalten umbenennen für den Join
df_de = df_de.withColumnRenamed("anzahl", "anzahl_deutsch")
df_nd = df_nd.withColumnRenamed("anzahl", "anzahl_nichtdeutsch")

# Inner Join über die gemeinsamen Schlüssel
df_pks = df_de.join(df_nd, on=["jahr", "schluessel", "straftat"], how="inner")

# Gesamtanzahl berechnen
df_pks = df_pks.withColumn("anzahl_gesamt", col("anzahl_deutsch") + col("anzahl_nichtdeutsch"))

# Einzelne Teildaten + kombiniertes Ergebnis speichern
df_de.write.mode("overwrite").parquet(f"{PROC_DIR}pks_deutsche_clean.parquet")
df_nd.write.mode("overwrite").parquet(f"{PROC_DIR}pks_nichtdeutsche_clean.parquet")
df_pks.write.mode("overwrite").parquet(f"{PROC_DIR}pks_complete.parquet")

print(f"pks_complete: {df_pks.count()} Zeilen")
df_pks.show(5)